# Obtén datos de la API de Marvel

![imagen](https://cdn.hobbyconsolas.com/sites/navi.axelspringer.es/public/styles/hc_1440x810/public/media/image/2021/09/marvel-2460339.jpg?itok=cBnC1CBi)

Te tendrás que [registrar](https://developer.marvel.com/) y consultar la [documentación](https://developer.marvel.com/documentation/generalinfo)

Queremos que consultes a la api para que te devuelva la info de los personajes de marvel que empiecen por la inicial de tu nombre. Deberemos guardarlos en un csv la información con la siguiente estructura

![imagen](./img/Captura_marvel.PNG)

Aquí te dejamos el código en python para poder empezar a hacer las llamadas, rellenando algunas variables como tus keys, parámetros de la llamada y la url (endpoint) a la que quieres acceder

In [28]:
# The Marvel Comics API’s base endpoint is https://gateway.marvel.com/
'''
You can access six resource types using the API:
- Comics:                       individual print and digital comic issues, collections and graphic novels. For example: 
                                Amazing Fantasy #15.
- Comic series:                 sequentially numbered (well, mostly sequentially numbered) groups comics with the same 
                                title. For example, Uncanny X-Men.
- Comic stories:                indivisible, reusable components of comics. For example, the cover from Amazing Fantasy 
                                #15 or the origin of Spider-Man story from that comic.
- Comic events and crossovers:  big, universe-altering storylines. For example, Infinity
- Creators:                     women, men and organizations who create comics. For example, Jack Kirby.
- Characters:                   the women, men, organizations, alien species, deities, animals, non-corporeal entities, 
                                trans-dimensional manifestations, abstract personifications, and green amorphous blobs 
                                which occupy the Marvel Universe (and various alternate universes, timelines and altered 
                                realities therein). For example, Spider-Man.

# Requests which fail authentication generally pass a 401 HTTP code and an error describing the reason for rejection.
# All endpoints currently accept only HTTP GET requests.
# If the content has not changed since the last request, the response code will return with an empty body 
# and a 304/Not Modified HTTP header and you can use a previously-stored value for the content.


Request Url: http://gateway.marvel.com/v1/public/comics

Request Method: GET
Params: {
  "apikey": "889a868fb6644430fb02676e31f72b5b",
  "ts": "19052025 151800",
  "hash": "my_hash"
}
Headers: {
  Accept: */*
}
'''

'\nYou can access six resource types using the API:\n- Comics:                       individual print and digital comic issues, collections and graphic novels. For example: \n                                Amazing Fantasy #15.\n- Comic series:                 sequentially numbered (well, mostly sequentially numbered) groups comics with the same \n                                title. For example, Uncanny X-Men.\n- Comic stories:                indivisible, reusable components of comics. For example, the cover from Amazing Fantasy \n                                #15 or the origin of Spider-Man story from that comic.\n- Comic events and crossovers:  big, universe-altering storylines. For example, Infinity\n- Creators:                     women, men and organizations who create comics. For example, Jack Kirby.\n- Characters:                   the women, men, organizations, alien species, deities, animals, non-corporeal entities, \n                                trans-dimensional mani

In [29]:
# Importamos librerias
import requests
import zipfile
import pandas as pd

In [30]:
# antes de nada necesitamos generar los datos para autenticarnos.
# al registrarse en marvel nos proporcionan una api key pública y una privada.

# en la web de marvel NO ofrecen información completa de la documentación, 
# pero hay otro lugar donde se encuentra toda la info:
# https://www.postman.com/alex-postman-workspace/marvel-api-workspace/documentation/989e351/marvel-api/8e19d6aa-7e1e-453

# en la documentación piden que para autenticarse es necesario un hash:
# Server-side applications must pass two parameters in addition to the apikey parameter:
# - ts - a timestamp (or other long string which can change on a request-by-request basis)
# - hash - a md5 digest of the ts parameter, your private key and your public key (e.g. md5(ts+privateKey+publicKey)

# con la librería hashlib creamos el hash:

import hashlib
import time

# API keys
public_key = "141fe804efbebf61085ae8ab71ecac08"
private_key = "6c4529e660118fd0ba679c942e53fbc3a6ad8395"

# Timestamp
ts = str(int(time.time()))

# Generar hash
to_hash = ts + private_key + public_key
hash_md5 = hashlib.md5(to_hash.encode()).hexdigest()

print(ts)
print(hash_md5)


1747693206
61664397460a033194265a1c2aed41d9


In [31]:
# Construir parámetros
params = {
    "nameStartsWith": "X",  # mi inicial
    "ts": ts,
    "apikey": public_key,
    "hash": hash_md5,
    "limit": 100
}

# Hacer la solicitud
url = "https://gateway.marvel.com/v1/public/characters"
response = requests.get(url, params=params)

# Comprobar los datos
data = response.json()
print(data)

# para saber cuándo estoy enviando la petición
import datetime
now = datetime.datetime.now()
print(now)


{'code': 'InvalidCredentials', 'message': 'The passed API key is invalid.'}
2025-05-20 00:20:10.482062


In [ ]:
######### SUPERHEROAPI ##########
# Como la API de Marvel estaba dando error ()
# https://superheroapi.com/
# hago lo mismo para otra web/API, aunque ésta no requiere autenticación, por lo que es mucho más sencillo

import requests
import pandas as pd

# Información para la llamada a la API
token = "8553ec4f9d6c7665e05d21d8eaba8efb"
search_term = "X"
url = f"https://superheroapi.com/api/{token}/search/{search_term}"

# Llamada a la API
response = requests.get(url)
data = response.json() # guardamos el JSON como Diccionario

In [27]:
print(f'\nDATA: {type(data)} \n {data}')

# Guardar los nombres en un dataframe
if data.get("response") == "success":
    results = data["results"] # lista dentro del JSON
    print(f'\nRESULTS: {type(results)} \n {results}')

    # Filtrar nombres que empiecen por el 'search_term'
    filtered = []
    for heroe in results:
        if heroe["name"].startswith(search_term): # heroe es un Diccionario
            print(f'\nHEROE: {type(heroe)} \n {heroe}')
            filtered.append(heroe) # añadimos a una lista 'filtered' cada coincidencia
    print(f'\nFILTERED: {type(filtered)} \n {filtered}')

    # Convierto los datos en una lista de diccionarios planos sólo con los ids, nombres y urls de las imágenes
    heroes_data = []
    for heroe in filtered: # Añado a la lista 'filtered' un diccionario sólo con los datos que me interesan
        heroes_data.append({
            "id": heroe["id"],
            "name": heroe["name"],
            "image_url": heroe["image"]["url"]
        })
    print(f'\nHEROES DATA: {type(heroes_data)} \n {heroes_data}')
    
    df = pd.DataFrame(heroes_data) # Los guardo en un DataFrame
    print(f'\nDATAFRAME: {type(df)} \n {df}')
else:
    print("\nNo se encontraron resultados o hubo un error.")

# Imprimo el resultado en un CSV con el punto y coma (';') como separador
df.to_csv("heroes_x.csv", sep=';', index=False)



DATA: <class 'dict'> 
 {'response': 'success', 'results-for': 'X', 'results': [{'id': '5', 'name': 'Abraxas', 'powerstats': {'intelligence': '88', 'strength': '63', 'speed': '83', 'durability': '100', 'power': '100', 'combat': '55'}, 'biography': {'full-name': 'Abraxas', 'alter-egos': 'No alter egos found.', 'aliases': ['-'], 'place-of-birth': 'Within Eternity', 'first-appearance': 'Fantastic Four Annual #2001', 'publisher': 'Marvel Comics', 'alignment': 'bad'}, 'appearance': {'gender': 'Male', 'race': 'Cosmic Entity', 'height': ['-', '0 cm'], 'weight': ['- lb', '0 kg'], 'eye-color': 'Blue', 'hair-color': 'Black'}, 'work': {'occupation': 'Dimensional destroyer', 'base': '-'}, 'connections': {'group-affiliation': 'Cosmic Beings', 'relatives': 'Eternity ("Father")'}, 'image': {'url': 'https://www.superherodb.com/pictures2/portraits/10/100/181.jpg'}}, {'id': '13', 'name': 'Ajax', 'powerstats': {'intelligence': '56', 'strength': '48', 'speed': '35', 'durability': '80', 'power': '34', 'com